# The Spanish Sentiment Model for Fine-Tuning

Using the translated and augmented spanish comments, we now fine tune a state-of-the-art sentiment model, trained on spanish language, with our data.

In [ ]:
!pip install --upgrade pip
!pip uninstall transformers -y #uninstall version you have currently, just bc we might need diff versions for diff fine tuning models
#!pip install transformers==4.18.0 #note that we need an older version of transformers to use this model
!pip install transformers 
!pip install --no-cache-dir transformers sentencepiece
!pip install datasets
!pip install accelerate -U

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import Dataset, load_metric
import torch
import pandas as pd
import numpy as np
from google.colab import files
from google.colab import drive
import glob
import zipfile
from sklearn.model_selection import train_test_split

In [2]:
def compute_metrics(p):
    metric = load_metric("accuracy")
    logits, labels = p
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [3]:
# Instantiate the tokenizer for the model
tokenizer = AutoTokenizer.from_pretrained("VerificadoProfesional/SaBERT-Spanish-Sentiment-Analysis")


def tokenize_function(examples, tokenizer=tokenizer):
    """
    Function to tokenize the data.
    examples : data to tokenize ; dict
    tokenizer : tokenizer to use ; DistilBertTokenizer
    """
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

In [4]:
def model(train_comments, train_labels,
                    val_comments, val_labels, test_comments,
                    test_labels,
                    batch_size_train, batch_size_val,
                    epochs, tokenizer=tokenizer, learning_rate=1e-4, weight_decay=0.001):
    """
    Function to train a  model on the data.
    train_comments : comments for training ; lst of str
    train_labels : labels for training ; lst of int
    val_comments : comments for validation ; lst of str
    val_labels : labels for validation ; lst of int
    batch_size_train : batch size for training ; int
    batch_size_val : batch size for validation ; int
    epochs : number of epochs ; int
    tokenizer : tokenizer to use
    """
    # Set the device to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = AutoModelForSequenceClassification.from_pretrained("VerificadoProfesional/SaBERT-Spanish-Sentiment-Analysis")
    model.to(device) 

    # Setup the Hugging Face Dataset Class
    train_dataset_dict = {"text": train_comments, "label": train_labels}
    val_dataset_dict = {"text": val_comments, "label": val_labels}
    test_dataset_dict = {"text": test_comments, "label": test_labels}

    train_dataset = Dataset.from_dict(train_dataset_dict)
    val_dataset = Dataset.from_dict(val_dataset_dict)
    test_dataset = Dataset.from_dict(test_dataset_dict)

    # Apply the tokenizer to the datasets
    train_dataset = train_dataset.map(tokenize_function, batched=True)
    val_dataset = val_dataset.map(tokenize_function, batched=True)
    test_dataset = test_dataset.map(tokenize_function, batched=True)

    # Remove columns we do not need for training
    train_dataset = train_dataset.remove_columns(["text"])
    val_dataset = val_dataset.remove_columns(["text"])
    test_dataset = test_dataset.remove_columns(["text"])

    # Set the format of the datasets to PyTorch tensors
    train_dataset.set_format("torch")
    val_dataset.set_format("torch")
    test_dataset.set_format("torch")


    # Training arguments
    training_args = TrainingArguments(
        output_dir='./results',          # output directory
        num_train_epochs=epochs,              # total number of training epochs
        per_device_train_batch_size=batch_size_train,  # batch size for training
        per_device_eval_batch_size=batch_size_val,   # batch size for evaluation
        warmup_steps=500,                # number of warmup steps for learning rate scheduler
        weight_decay=weight_decay,               # strength of weight decay
        logging_dir='./logs',            # directory for storing logs
        logging_steps=10,
        evaluation_strategy="steps",     # Evaluate every `eval_steps`
        eval_steps=10,                   # Number of steps between evaluations
        save_steps=10,                   # Save the model every `save_steps`
        load_best_model_at_end=True,     # Load the best model at the end of training#
        learning_rate= learning_rate,              # Set the learning rate
        metric_for_best_model="eval_loss", # Use evaluation loss to check how good our model is performing
        greater_is_better=False,
    )

    # Trainer
    trainer = Trainer(
        model=model,                         # model
        args=training_args,                  # training arguments, defined above
        train_dataset=train_dataset,         # training dataset
        eval_dataset=val_dataset,            # evaluation dataset
        tokenizer=tokenizer,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=10)], # Early Stopping for Overfitting
        compute_metrics=compute_metrics,
    )

    # Train the model
    trainer.train()

    # Evaluate the model on the test set
    test_results = trainer.evaluate(eval_dataset=test_dataset)

    # Get test accuracy
    print("Test Accuracy:", test_results['eval_accuracy'])



    return model, tokenizer, test_results



In [5]:
def save_model(model, tokenizer, path):
    """
    Function to save the model
    model : model to save ;
    tokenizer : tokenizer to save ;
    path : path to save the model ; str
    """

    model_save_path =  path
    model.save_pretrained(model_save_path)
    tokenizer.save_pretrained(model_save_path)

In [ ]:
# Open a file upload dialog
# Select here all files to upload!
# If already uploaded, just press 'Cancel Upload'
# They are contained within a zip file.
uploaded = files.upload()

In [ ]:
# Set the path to the data
# On local machine use the relative path, for example
# path = 'NLP labelled data preview/english set/'
# On Google Colab use this path
# '/content/'
path = '/content/'

In [ ]:
all_comments_with_labels = pd.read_csv(path + 'spanish_combined_ready_for_finetuning.csv')

In [7]:
# Conversion to labels of the model we fine-tune
# model stores 0 as negative, 1 as positive 
# since we store 0 as negative and 2 as positive, we have to change accordingly
all_comments_with_labels['Label'] = all_comments_with_labels['Label'].apply(lambda x: 0 if x == 0 else 1)

# Print the first few rows to check the changes
print(all_comments_with_labels.head())

# Check the label count
print(all_comments_with_labels['Label'].value_counts())

   Unnamed: 0  index  Label     Score  \
0           0      0      0  0.865466   
1           1      1      0  0.865466   
2           2      2      0  0.865466   
3           3      3      0  0.865466   
4           4      4      0  0.865466   

                                     Comment  
0  miedo demócratas algún humano creado ai    
1                Me temo demócratas humanos   
2          Me temo demócratas seres humanos   
3                Me temo demócratas humanos   
4        Me temo demócratas creación humana   
0    815
2    594
Name: Label, dtype: int64


In [8]:
# Seperate the two columns in the dataframe into comments and labels and turn them into lists
comments = all_comments_with_labels['Comment'].tolist()
labels = all_comments_with_labels['Label'].tolist()


In [9]:
# Convert the labels to integers
labels = [int(label) for label in labels]
# Turn all elements in comments into strings
comments = [str(comment) for comment in comments]


In [11]:
# Split the data into a train and test set with stratification
train_comments, temp_comments, train_labels, temp_labels = train_test_split(
    comments, labels, test_size=0.3, random_state=42, stratify=labels)



In [12]:
# Split the data into training and validation sets with stratification
val_comments, test_comments, val_labels, test_labels = train_test_split(temp_comments, temp_labels, test_size=0.5, random_state=42, stratify= temp_labels)

In [13]:
# Look at the number of comments in the training and validation sets
print("Number of comments in the training set: ", len(train_comments))
print("Number of comments in the validation set: ", len(val_comments))
print("Number of comments in the test set: ", len(test_comments))
# Look at labels in the training and validation sets
print("Number of negative comments in the training set: ", train_labels.count(0))
print("Number of positive comments in the training set: ", train_labels.count(1))
print("Number of negative comments in the validation set: ", val_labels.count(0))
print("Number of positive comments in the validation set: ", val_labels.count(1))
print("Number of negative comments in the test set: ", test_labels.count(0))
print("Number of positive comments in the test set: ", test_labels.count(1))

Number of comments in the training set:  986
Number of comments in the validation set:  380
Number of comments in the test set:  43
Number of negative comments in the training set:  570
Number of positive comments in the training set:  416
Number of negative comments in the validation set:  220
Number of positive comments in the validation set:  160
Number of negative comments in the test set:  25
Number of positive comments in the test set:  18


In [ ]:
# Train the model
model_trained, tokenizer_trained, test_results = model(train_comments, train_labels, val_comments, val_labels,test_comments, test_labels, batch_size_train = 128, batch_size_val = 128, epochs = 2, tokenizer = tokenizer, learning_rate=1e-4, weight_decay=0.001)


In [ ]:
# Save the model
save_model(model_trained, tokenizer_trained, path + 'spanish_model')


In [ ]:
# Save the fine-tuned model to your system

# Mount to drive
drive.mount('/content/drive')

!cp -r /content/spanish_model /content/drive/MyDrive/

# Now download it from your Google Drive Account !